In [98]:
import requests
import json
import zipfile
import zlib
from io import BytesIO
import pandas as pd 
import numpy as np

In [3]:
def df_statscan(table_id):
    url = f"https://www150.statcan.gc.ca/t1/wds/rest/getFullTableDownloadCSV/{table_id}/en"
    r = requests.get(url)
    download_url = json.loads(r.content)['object']
    r = requests.get(download_url)
    if r.status_code == 200:
        csvzipbytes = r.content
        csvzip = zipfile.ZipFile(BytesIO(csvzipbytes))
        csvcontent = csvzip.read(csvzip.namelist()[0])
        df = pd.read_csv(BytesIO(csvcontent))
        return df

In [150]:
df_raw = df_statscan('33100270') # input table number 

In [153]:
df = df_raw.copy(deep=False)

# Data cleaning and pre-processing

# Get only relevant columns
df = df[['REF_DATE', 'GEO', 'Industry', 'Business dynamics measure', 'VALUE']].copy(deep=False)

# Rename columns 
df.columns = ['Date', 'Geography', 'Industry', 'Business dynamics measure', 'Number of businesses']

# Remove non-relevant strings for purpose of analysis 
df['Industry'] = df['Industry'].str.replace(r"\[.*\]", '', regex=True).str.strip()

# Remove "\xa0" from Geography column
df['Geography'] = df['Geography'].str.split().str.join(' ')

# Create Province column
df['Province'] = df['Geography'].str.rsplit(', ').str[-1] 

# Remove 'Canada' in Geography column
df = df[df.Geography != 'Canada']

In [154]:
df['Date'] =  pd.to_datetime(df['Date'], format='%Y-%m')
df['Date'] = df['Date'] + pd.offsets.MonthEnd(0) 
df

,Date,Geography,Industry,Business dynamics measure,Number of businesses,Province
256,2015-01-31,Newfoundland and Labrador,Business sector industries,Active businesses,14319.0,Newfoundland and Labrador
257,2015-01-31,Newfoundland and Labrador,Business sector industries,Opening businesses,746.0,Newfoundland and Labrador
258,2015-01-31,Newfoundland and Labrador,Business sector industries,Continuing businesses,13591.0,Newfoundland and Labrador
259,2015-01-31,Newfoundland and Labrador,Business sector industries,Closing businesses,692.0,Newfoundland and Labrador
260,2015-01-31,Newfoundland and Labrador,Business sector industries,Reopening businesses,439.0,Newfoundland and Labrador
...,...,...,...,...,...,...
1199611,2022-12-31,Nunavut,Business sector industries (except educational...,Closing businesses,NaN,Nunavut
1199612,2022-12-31,Nunavut,Business sector industries (except educational...,Reopening businesses,NaN,Nunavut
1199613,2022-12-31,Nunavut,Business sector industries (except educational...,Entrants,NaN,Nunavut
1199614,2022-12-31,Nunavut,Business sector industries (except educational...,Temporary closures,NaN,Nunavut


In [155]:
df['Industry'].unique()

array(['Business sector industries', 'Forestry, fishing and hunting',
       'Mining, quarrying, and oil and gas extraction', 'Utilities',
       'Construction', 'Manufacturing', 'Food manufacturing',
       'Beverage and tobacco product manufacturing', 'Wholesale trade',
       'Retail trade', 'Transportation and warehousing',
       'Information and cultural industries',
       'Finance and insurance and Management of companies and enterprises',
       'Real estate and rental and leasing',
       'Professional, scientific and technical services',
       'Administrative and support, waste management and remediation services',
       'Educational services', 'Health care and social assistance',
       'Arts, entertainment and recreation',
       'Accommodation and food services',
       'Other services (except public administration)',
       'Tourism industry', 'Total tourism transportation',
       'Air transportation (tourism)',
       'Rail, scenic and sightseeing transportation (tou

In [156]:
df['Business dynamics measure'].unique()

array(['Active businesses', 'Opening businesses', 'Continuing businesses',
       'Closing businesses', 'Reopening businesses', 'Entrants',
       'Temporary closures', 'Exits'], dtype=object)

In [126]:
df.to_csv('/Downloads/Canada_Business_Dynamics_raw.csv', index=False)